In [90]:
import torch

# trajs = torch.load("/home/yifeizhou/logs/jack-trajectory+mc-newenv-online-general-step10-1-offlinenotraj/trajectories.pt")
# trajs = torch.load("/home/yifeizhou/logs/jack-trajectory+mc-newenv-online-general-step10-2-offlinenotraj/trajectories.pt")
trajs = torch.load("/home/yifeizhou/logs/jack-filteredbc-general-train-online-step10-1/trajectories.pt")
# trajs = torch.load("/home/yifeizhou/logs/cogagent/trajectories_eval.pt")

# print successful trajectories in the last 100 trajs
# successful_traj_ids = []
# for i, traj in enumerate(trajs):
#     if traj[-1]['reward'] == 1:
#         successful_traj_ids.append(i)
# print(successful_traj_ids[-100:])

# print id of trajectory with task "restaurant in Las Vegas"
for i, traj in enumerate(trajs):
    if "Search for flights from Helsinki to Seoul" in traj[0]['task']:
        print(i)

3
83
165
886
922
942
1474
2050


In [93]:
import re

def parse_action(action):
    # the given actions is a string look like 'Action Plan: ... ; Action Decision: "action_type": "...", "touch_point": "...", "lift_point": "...", "typed_text": "..."'
    fill = ["action_type", "touch_point", "lift_point", "typed_text"]
    action_dict = {}
    for key in fill:
        # extract the value of each key in the action (exmaple shown in template)
        pattern = f'"{key}": "(.*?)"'
        match = re.search(pattern, action)
        action_dict[key] = match.group(1) if match else None
    
    return action_dict

In [96]:
from PIL import Image, ImageDraw, ImageFont

traj_id = 1474
dir_name = str(traj_id)
# make dir
import os
if not os.path.exists("images/" + dir_name):
    os.makedirs("images/" + dir_name)
image_paths = []
actions = []
for step in trajs[traj_id]:
    image_paths.append(step['image_path'])
    actions.append(step['action'])

# make new images, each image is the concatenation of 3 images: this image, annotated this image, and next image
for i in range(len(image_paths) - 1):
    img1 = Image.open(image_paths[i])

    total_width = img1.width * 3
    action_annotation_height = 550
    max_height = img1.height + action_annotation_height
    row_image = Image.new('RGB', (total_width, max_height))
    # make background white
    row_image.paste((255, 255, 255), (0, 0, total_width, max_height))
    print("row image size:", row_image.size)
    row_image.paste(img1, (0, 0))

    img1_annotated = img1.copy().convert("RGB")
    draw = ImageDraw.Draw(img1_annotated)
    parse = "True"
    if parse == "True":
        action_dict = parse_action(actions[i+1])
        action = "\n".join([f"{key}: {value}" for key, value in action_dict.items()])
        if action_dict['action_type'] == 'DUAL_POINT':
            import ast
            touch_point_ratio = ast.literal_eval(action_dict['touch_point'])
            lift_point_ratio = ast.literal_eval(action_dict['lift_point'])
            touch_point = (int(touch_point_ratio[0] * img1_annotated.height), int(touch_point_ratio[1] * img1_annotated.width))
            lift_point = (int(lift_point_ratio[0] * img1_annotated.height), int(lift_point_ratio[1] * img1_annotated.width))
            if touch_point[0] == lift_point[0] and touch_point[1] == lift_point[1]:
                # draw a large red point for the touch point
                x = touch_point[1]
                y = touch_point[0]
                r = 20  # radius of the circle
                draw.ellipse([(x - r, y - r), (x + r, y + r)], fill=(255, 0, 0))
            else:
                # draw an arrow between the two points
                draw.line([touch_point, lift_point], fill=128, width=2, joint='curve')
    else:
        # break the action string into multiple lines
        action = actions[i+1]
        grounded_opration = action.split("Grounded Operation:")[1]
        # split into words by space then reserve at most 35 characters per line
        string_split_len = 130
        action_words = action.split()
        action = []
        while action_words:
            line = ""
            while action_words and len(line) + len(action_words[0]) <= string_split_len:
                line += action_words.pop(0) + " "
            action.append(line)
        action = "\n".join(action)
        if "tap" in grounded_opration:
            regex = r"tap [[0-9]+,[0-9]+]"
            # extract x and y from the tap operation
            match = re.search(regex, grounded_opration)
            if match:
                x, y = match.group(0).split(" ")[1].replace("[", "").replace("]", "").split(",")
                x, y = int(x) / 1000 * img1_annotated.width, int(y) / 1000 * img1_annotated.height
                # draw a large red point for the touch point
                r = 20
                draw.ellipse([(x - r, y - r), (x + r, y + r)], fill=(255, 0, 0))
    # action annotation
    draw = ImageDraw.Draw(row_image)
    draw.text((100, img1.height), "State " + str(i), fill=(0, 0, 0), font=ImageFont.load_default(90))
    draw.text((100+img1.width, img1.height), "Action" + str(i), fill=(0, 0, 0), font=ImageFont.load_default(90))
    if parse == "False":
        draw.text((100, img1.height+100), str(action), fill=(0, 0, 0), font=ImageFont.load_default(50))
    else:
        draw.text((100+img1.width, img1.height+100), str(action), fill=(0, 0, 0), font=ImageFont.load_default(50))
    draw.text((100+2*img1.width, img1.height), "State " + str(i+1), fill=(0, 0, 0), font=ImageFont.load_default(90))

    row_image.paste(img1_annotated, (img1.width, 0))

    img2 = Image.open(image_paths[i + 1])
    row_image.paste(img2, (img1.width * 2, 0))

    # save the row image to a file with step number as the name (1.png, 2.png, ...)
    row_image.save(f"images/{dir_name}/{i}.png")

    # # print
    # import matplotlib.pyplot as plt
    # plt.imshow(row_image)
    # # remove axis
    # plt.axis('off')
    # plt.show()

row image size: (3240, 2830)
row image size: (3240, 2830)
row image size: (3240, 2830)
row image size: (3240, 2830)
